In [ ]:
import requests
import datetime
!pip install pyodbc
import pyodbc
import holidays

  Using cached pyodbc-5.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.7 kB)
Using cached pyodbc-5.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (334 kB)


In [ ]:
#Informações para conectar-se ao banco

SERVER = 'PLACEHOLDER'
DATABASE = 'PLACEHOLDER'
USERNAME = 'PLACEHOLDER'
PASSWORD = 'PLACEHOLDER'

In [ ]:
#Estabelecendo conexão

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

conn = pyodbc.connect(connectionString)

In [ ]:
#Requisitando informações da API

# link do open_weather: https://openweathermap.org/
# https://openweathermap.org/api/one-call-3

API_KEY = "5643ad2c732eb521a8e7cd2723c3369c"
cidade = "são vicente"
link = f"https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={API_KEY}&lang=pt_br"

requisitions = requests.get(link)
informations = requisitions.json()

In [ ]:
#Traduzindo o dia da semana

def get_weekday(time_info):
  #currentTime = datetime.now().strftime("%H:%M:%S")
  weekDay = datetime.date.today().weekday()

  day = ["Segunda-feira", "Terça-feira", "Quarta-feira", "Quinta-feira", "Sexta-feira", "Sabado", "Domingo"]

  return day[weekDay]

#Traduzindo o mês

def get_month(time_info):

  month = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

  return month[time_info.month]

#Hoje é feriado?

def get_holiday(time_info):
  feriado = holidays.country_holidays("BR", subdiv="SP")

  feriados_ano = feriado[(str(time_info.year) + '-' + "1" + '-' "1"): (str(time_info.year) + '-' + "12" + '-' "31")]
  data = str(time_info.year) + '-' + str(time_info.month) + '-' + str(time_info.day)
  for feriado in feriados_ano:
    if(data == feriado):
      return True
  return False

In [ ]:
time_info = datetime.datetime.now()

dados = {
    "dia_semana": get_weekday(time_info),
    "dia_mes": str(time_info.day),
    "mes": get_month(time_info),
    "ano": time_info.year,
    "clima": informations['weather'][0]['description'],
    "feriado": get_holiday(time_info),
    "temperatura_med": round((((round(informations['main']['temp_max']-273, 2)) + (round(informations['main']['temp_min']-273, 2)))/2),2),
    "sensacao_termica": str(round(informations['main']['feels_like']-273, 1)),
    "umidade": str(informations['main']['humidity']),
    "velocidade_vento": str(round(informations['wind']['speed']*3.6, 2)),
}

In [ ]:
dados

{'dia_semana': 'Terça-feira',
 'dia_mes': '17',
 'mes': 'Outubro',
 'ano': 2024,
 'clima': 'névoa',
 'feriado': False,
 'temperatura_med': 19.63,
 'sensacao_termica': '18.9',
 'umidade': '26',
 'velocidade_vento': '7.42'}

In [ ]:
#Inserindo informações no banco

SQL_STATEMENT = """

INSERT INTO dbo.tb_noa (dia_semana, dia_mes, mes, ano, clima, feriado, temperatura, umidade, velocidade)
VALUES (dados["dia_semana"], dados["dia_mes"], dados["mes"], dados["ano"], dados["clima"], dados["feriado"], dados["temperatura_med"], dados["umidade"], dados["velocidade_vento"]);

"""

SQL_QUERY = """

SELECT * FROM tb_noa

"""

cursor = conn.cursor()

cursor.execute(SQL_STATEMENT)

cursor.execute(SQL_QUERY)

records = cursor.fetchall()
for r in records:
    print(r)

conn.commit()
cursor.close()
conn.close()